# Initialisation: Import `.gh` files

In [1]:
import sys
from pathlib import Path

# Find the nearest directory that has a "src" folder (works no matter where you run the notebook)
here = Path.cwd()
src_dir = next(p for p in [here, *here.parents] if (p / "src").exists()) / "src"
sys.path.insert(0, str(src_dir))

In [2]:
# a block primarily for handling `.idf` files from `Rhino + Grasshopper` via `Ladybug Tools`
from ReIDF import relocate_IDF, md_ghIDF, md_IDFsch2series, md_IDFsch4gh, md_IDFdt, md_IDFop_ctr

source_directory_1 = R"C:\Users\s2589602\0.Recents\1.Rhino\GH-outputs\HVACDetailed-Oct-15\openstudio\run"
dest_folder_name='idf-Oct'

paths = [] # KEY-1
for source_directory in [source_directory_1]:
    path=relocate_IDF(source_directory, dest_folder_name)
    md_ghIDF(path, year=2023)
    
    # Universal functions
    md_IDFsch2series(path)
    md_IDFsch4gh(path)
    md_IDFdt(path, '2023-01-01', '2023-12-31')
    md_IDFop_ctr(path, ['CSV', 'MTR', 'Tabular'])

    paths.append(path) # KEY-2

File copied and renamed to:
c:\Users\s2589602\Downloads\2025\src\eplus_heat_balance\idf-Oct\HVACDetailed-Oct-15.idf
idf-Oct/HVACDetailed-Oct-15.idf
Modified idf-Oct/HVACDetailed-Oct-15.idf


In [3]:
path

'idf-Oct/HVACDetailed-Oct-15.idf'

# Initialisation: Revision Record

In [4]:
from ReIDF import ReIDF1

# clg_vrf_updated_='23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23;'
# clg_office_updated_='35, 35, 35, 35, 35, 35, 35, 35, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 35, 35, 35, 35, 35;'
# clg_con_office_updated_='35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35;'
# 0, 0, 0, 0, 0, 0, 0.07, 0.19, 0.35, 0.38, 0.39, 0.47, 0.57, 0.54, 0.34, 0.33, 0.44, 0.26, 0.21, 0.15, 0.17, 0.08, 0.05, 0.05;
# 0, 0, 0, 0, 0, 0, 0.07, 0.11, 0.15, 0.21, 0.19, 0.23, 0.2, 0.19, 0.15, 0.13, 0.14, 0.07, 0.07, 0.07, 0.07, 0.09, 0.05, 0.05;

ReClass_HVAC = [
    # ('ZoneVentilation:WindandStackOpenArea', 3, 2),
    # ('ZoneVentilation:WindandStackOpenArea', 7, 2),
    ("ZoneInfiltration:DesignFlowRate", 7, 0.00022),
    # ('Fan:OnOff', 5, 0.33),
    # ('Fan:ConstantVolume', 5, 0.2163)
]
#0.4153
ReUnit_HVAC = [
    ('inftr_atrium_br_', {4:'AirChanges/Hour', 8:2}),
    # ({'Coil:Heating:Water': '26 Zone DOAS Htg Coil'}, {3: '', 4: '', 9: 'NominalCapacity', 10: 150000}),
    # ({'Coil:Heating:Water': '35 Zone DOAS Htg Coil'}, {3: '', 4: '', 9: 'NominalCapacity', 10: 155000}),
    # ({'Coil:Heating:Water': '1 Zone DOAS Htg Coil'}, {3: '', 4: '', 9: 'NominalCapacity', 10: 49000}),
    # ({'Coil:Heating:Water': '63 Zone DOAS Htg Coil'}, {3: '', 4: '', 9: 'NominalCapacity', 10: 155000}),
    # ({'Fan:ConstantVolume':'DOAS Exhaust Fan'}, 5, 0.4153), 
    # ({'Fan:ConstantVolume':'DOAS Exhaust Fan 1'}, 5, 0.4153), 
    # ({'Fan:ConstantVolume':'DOAS Exhaust Fan 2'}, 5, 0.4153),    
    # ('clg_sch_office_br__Hol 1',3, f'{clg_con_office_updated_}'),         
]
updated_idf = ReIDF1(
    oriPath=path,
    ReTemp=None,
    ReClass=ReClass_HVAC,
    ReUnit=ReUnit_HVAC,
)

ReIDF: → C:\Users\s2589602\Downloads\2025\src\eplus_heat_balance\idf-Oct\GEN_ReIDF\HVACDetailed-Oct-15_modified_3.idf
ReUnit: [{'inftr_atrium_br_': 2}]
ReClass: [{'ZoneInfiltration:DesignFlowRate': 14}]


# 2. Executing EnergyPlus
* For `mac` operations, using files:

    weather_file = '/Users/rui.bo/Downloads/EnergyPlus/GBR_SCT_Edinburgh.AP.031600_TMYx.2023.epw'

    energyplus_exe = '/Applications/EnergyPlus-23-2-0/EnergyPlus'

* For `windows` operations, using files:

    energyplus_exe = 'C:/EnergyPlusV23-2-0/EnergyPlus'

    weather_file = 'C:/Users/s2589602/Downloads/a2-May/EPlus/EPlus/epw/GBR_SCT_Edinburgh.AP.031600_TMYx.2023.epw'

(1) Single File

In [5]:
from ReIDF import run_IDF

weather_file = '../_base/epw/ED-TMYx.2023.epw'
energyplus_exe = 'C:/EnergyPlusV23-2-0/EnergyPlus'

output_folder = run_IDF(updated_idf, weather_file, energyplus_exe)

Simulation completed successfully.
Output Folder: C:\Users\s2589602\Downloads\2025\src\eplus_heat_balance\idf-Oct\GEN_ReIDF\HVACDetailed-Oct-15_modified_3


In [8]:
from pathlib import Path
import re
import pandas as pd

def relabel_energyplus(
    csv_in,
    csv_out=None,
    mode="calendar",          # "calendar" or "index"
    start="2023-01-01",       # used in calendar mode
    freq="H",                 # e.g. "H" (hourly), "10min", "30min"
    expected_periods=None,    # e.g. 8760, 8784, or 365*24
    keep_original_datetime=False  # set True to keep original "Date/Time"
):
    """
    Read an EnergyPlus CSV/MTR, drop the text 'Date/Time' column, and add a user-defined timestamp.
    If expected_periods is given and the file has extra rows (e.g., design/warmup days),
    keep only the last `expected_periods` rows.
    """

    csv_in = Path(csv_in)

    # --- Detect the header row + encoding robustly (handles utf-8-sig BOM, cp1252, comments, spaces) ---
    def _find_header_row_and_encoding(path: Path):
        # Try common encodings EnergyPlus uses
        for enc in ("utf-8-sig", "utf-8", "cp1252"):
            try:
                with path.open("r", encoding=enc, errors="ignore") as f:
                    for i, raw in enumerate(f):
                        line = raw.replace("\ufeff", "").strip()
                        if not line or line.startswith("!"):
                            continue
                        # Tokenize and normalize quotes/spaces
                        tokens = [t.replace('"', "").strip() for t in line.split(",")]
                        if any(t.lower().replace(" ", "") == "date/time" for t in tokens):
                            return i, enc
            except Exception:
                # try next encoding
                continue
        raise ValueError("Could not find a header row containing 'Date/Time'.")

    header_row, encoding = _find_header_row_and_encoding(csv_in)

    # --- Read using the detected header/encoding ---
    df = pd.read_csv(
        csv_in,
        header=header_row,
        encoding=encoding,
        engine="python",      # more tolerant
        comment="!",          # ignore trailing EnergyPlus comments if present
        skipinitialspace=True # trim spaces after commas
    )

    # Standardize column names (strip spaces/quotes around names)
    df.columns = [c.replace('"', "").strip() for c in df.columns]

    # --- Optionally trim to expected periods (drop warmup/design days) ---
    if expected_periods is not None:
        if len(df) >= expected_periods:
            df = df.tail(expected_periods).copy()
        else:
            raise ValueError(
                f"File has only {len(df)} rows, fewer than expected_periods={expected_periods}."
            )

    # --- Drop the original Date/Time text column (optional keep) ---
    # Some files have leading spaces on the first timestamp cell—dropping by column name is safer.
    dt_cols = [c for c in df.columns if c.lower().replace(" ", "") == "date/time"]
    if dt_cols and not keep_original_datetime:
        df = df.drop(columns=dt_cols)

    # --- Build the new timestamp/index ---
    if mode == "index":
        new_ts = pd.RangeIndex(start=0, stop=len(df), step=1)
        df.insert(0, "ts", new_ts)
    elif mode == "calendar":
        periods = len(df)
        new_ts = pd.date_range(start=start, periods=periods, freq=freq)
        df.insert(0, "timestamp", new_ts)
    else:
        raise ValueError("mode must be 'calendar' or 'index'.")

    # --- Save or return ---
    if csv_out:
        csv_out = Path(csv_out)
        df.to_csv(csv_out, index=False, encoding="utf-8")
        return csv_out
    return df


def compute_eui_table(
    df: pd.DataFrame,
    area_m2: float,
    include_patterns=None,   # e.g. [r"Electricity", r"District"]
    exclude_patterns=None,   # e.g. [r"Meter|Submeter"]
    require_hourly_joules=True,   # only columns like "... [J](Hourly)"
    add_total=True
) -> pd.DataFrame:
    """
    Auto-detect energy columns and compute annual energy (kWh) + EUI (kWh/m²·yr).
    Expects per-timestep energy in Joules (e.g., '[J](Hourly)').
    """
    J_TO_KWH = 1 / 3_600_000

    # 1) pick candidates from header patterns
    cols = list(df.columns)

    def _match(c: str, patterns):
        return any(re.search(p, c, flags=re.IGNORECASE) for p in patterns)

    candidates = []
    for c in cols:
        if c in ("timestamp", "ts"):
            continue
        if require_hourly_joules:
            if not re.search(r"\[J\]\s*\(Hourly\)", c, flags=re.IGNORECASE):
                continue
        if include_patterns and not _match(c, include_patterns):
            continue
        if exclude_patterns and _match(c, exclude_patterns):
            continue
        candidates.append(c)

    # 2) convert to numeric safely and compute
    out = []
    for c in candidates:
        s = pd.to_numeric(df[c], errors="coerce")
        if s.notna().sum() == 0:
            continue  # skip non-numeric cols
        annual_J = s.sum(skipna=True)
        annual_kWh = annual_J * J_TO_KWH
        eui = annual_kWh / area_m2 if area_m2 > 0 else float("nan")
        out.append({"Metric": c, "Annual Energy (kWh)": annual_kWh, "EUI (kWh/m²·yr)": eui})

    res = pd.DataFrame(out).sort_values("Metric").reset_index(drop=True)

    # 3) optional total
    if add_total and not res.empty:
        total_kWh = res["Annual Energy (kWh)"].sum()
        total_eui = total_kWh / area_m2 if area_m2 > 0 else float("nan")
        res.loc[len(res)] = {"Metric": "TOTAL", "Annual Energy (kWh)": total_kWh, "EUI (kWh/m²·yr)": total_eui}

    return res


# --- Example usage ---
from pathlib import Path
ot_mtr = Path(str(Path(output_folder).with_suffix("")) + "/eplusmtr.csv")

df = relabel_energyplus(ot_mtr,
                        csv_out=None, mode="calendar",
                        start="2023-01-01", freq="H",
                        expected_periods=365*24)
area_m2 = 9600
eui_table = compute_eui_table(df, area_m2)
eui_table 

C:\Users\s2589602\AppData\Local\Temp\ipykernel_8068\1697683578.py:77: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  new_ts = pd.date_range(start=start, periods=periods, freq=freq)


,Metric,Annual Energy (kWh),EUI (kWh/m²·yr)
0,Baseboard:EnergyTransfer [J](Hourly),1.944537e+05,20.255598
1,Baseboard:PlantLoopHeatingDemand [J](Hourly),1.944537e+05,20.255598
2,Cooling:Electricity [J](Hourly),5.219206e+04,5.436673
3,CoolingCoils:EnergyTransfer [J](Hourly),1.762909e+05,18.363640
4,DistrictCooling:Facility [J](Hourly),0.000000e+00,0.000000
5,DistrictHeatingWater:Facility [J](Hourly),2.192595e+05,22.839533
6,Electricity:Facility [J](Hourly),6.543852e+05,68.165127
7,Electricity:HVAC [J](Hourly),5.704136e+04,5.941808
8,Electricity:Plant [J](Hourly),8.955661e+04,9.328813
9,HeatingCoils:EnergyTransfer [J](Hourly),2.624664e+04,2.734025
